In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import os
# Scale data
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline

c:\python35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
peilvak_in = 'peilvak.92'
color_peilvak = 'orange'
pv_in_csv = r'D:\Projects\RO\0815.10 data Challenge 2018\Neerslag//{}.csv'.format(peilvak_in)
png_out = r'D:\Projects\RO\DataChallenge2018\png_out\waterlevel.{}'.format(peilvak_in)

# Import df
df = pd.read_csv(pv_in_csv)
df.set_index('datetime', inplace=True)
df.index = pd.to_datetime(df.index)

# maak een month_day dataframe van de MultiYear DataFrame
month_day = pd.concat([
                df.index.to_series().dt.month, 
                df.index.to_series().dt.day
            ], axis=1).apply(tuple, axis=1)        

In [3]:
#zomerpeil = 0.4

In [4]:
#df['Y.waterlevel'] = df['Y.waterlevel'] - zomerpeil

In [5]:
df.tail()

,X.last1Hsum,X.last2Hsum,X.last4Hsum,X.last8Hsum,X.last16Hsum,X.last32Hsum,X.coming1Hsum,X.coming2Hsum,X.coming3Hsum,X.coming4Hsum,X.coming5Hsum,X.coming6Hsum,Y.waterlevel
datetime,,,,,,,,,,,,,
2017-12-31 20:00:00,0.0533,3.4094,10.0865,14.3265,16.4529,25.1766,0.0000,0.0996,0.1352,0.1352,NaN,NaN,NaN
2017-12-31 21:00:00,0.0000,0.0533,7.9786,13.8284,14.8296,25.1766,0.0996,0.1352,0.1352,NaN,NaN,NaN,NaN
2017-12-31 22:00:00,0.0996,0.0996,3.5090,12.7889,14.6309,25.2762,0.0356,0.0356,NaN,NaN,NaN,NaN,NaN
2017-12-31 23:00:00,0.0356,0.1352,0.1885,11.5726,14.6665,25.3118,0.0000,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-01 00:00:00,0.0000,0.0356,0.1352,10.2217,14.6665,25.3118,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# df = df[['X.last1Hsum', 'X.last2Hsum', 'X.last4Hsum', 'X.last8Hsum', 'X.last16Hsum', 'X.last24Hsum',
#     'X.coming1Hsum', 'X.coming2Hsum', 'X.coming3Hsum', 'X.coming4Hsum', 'X.coming5Hsum', 'X.coming6Hsum',
#     'Y.waterlevel']]

In [7]:
df.fillna(0, inplace=True)

In [8]:
# before_2015 = df['2010-01-01':'2014-12-31']
# during_2015 = df['2015-01-01':'2015-12-31']
# after_2015 = df['2016-01-01':'2018-12-31']

In [9]:
train_df = df#pd.concat([before_2015, during_2015, after_2015], axis=0)
#train_df = during_2015
#test_df = during_2015

In [10]:
train_df['Y.waterlevel.FC.1H'] = train_df['Y.waterlevel'].shift(freq=pd.Timedelta('-1 hour'))
train_df['Y.waterlevel.FC.2H'] = train_df['Y.waterlevel'].shift(freq=pd.Timedelta('-2 hour'))
train_df['Y.waterlevel.FC.3H'] = train_df['Y.waterlevel'].shift(freq=pd.Timedelta('-3 hour'))
train_df['Y.waterlevel.FC.4H'] = train_df['Y.waterlevel'].shift(freq=pd.Timedelta('-4 hour'))
train_df['Y.waterlevel.FC.5H'] = train_df['Y.waterlevel'].shift(freq=pd.Timedelta('-5 hour'))
train_df['Y.waterlevel.FC.6H'] = train_df['Y.waterlevel'].shift(freq=pd.Timedelta('-6 hour'))

In [11]:
# selecteer alleen de gebeurtenissen binnen een jaarlijks terugkerende periode 
startMM = 4
startdd = 1
endMM = 9
enddd = 30
train_df = train_df[(month_day >= (startMM, startdd)) & (month_day <= (endMM, enddd))]
test_df = train_df['2015-01-01':'2015-12-31']
train_df.tail()

,X.last1Hsum,X.last2Hsum,X.last4Hsum,X.last8Hsum,X.last16Hsum,X.last32Hsum,X.coming1Hsum,X.coming2Hsum,X.coming3Hsum,X.coming4Hsum,X.coming5Hsum,X.coming6Hsum,Y.waterlevel,Y.waterlevel.FC.1H,Y.waterlevel.FC.2H,Y.waterlevel.FC.3H,Y.waterlevel.FC.4H,Y.waterlevel.FC.5H,Y.waterlevel.FC.6H
datetime,,,,,,,,,,,,,,,,,,,
2017-09-30 19:00:00,0.0,1.271205e-14,2.116000e-01,1.4026,1.727,4.7453,0.0,1.271205e-14,-8.215650e-15,3.108624e-14,-1.465494e-14,6.128431e-14,-1.19200,-1.19200,-1.19200,-1.19225,-1.192,-1.192,-1.1950
2017-09-30 20:00:00,0.0,1.271205e-14,3.108624e-14,1.0244,1.727,4.7453,0.0,1.271205e-14,-8.215650e-15,3.108624e-14,-1.465494e-14,6.128431e-14,-1.19200,-1.19200,-1.19225,-1.19200,-1.192,-1.195,-1.1950
2017-09-30 21:00:00,0.0,1.271205e-14,3.108624e-14,0.8505,1.727,4.7453,0.0,1.271205e-14,-8.215650e-15,3.108624e-14,-1.465494e-14,6.128431e-14,-1.19200,-1.19225,-1.19200,-1.19200,-1.195,-1.195,-1.1950
2017-09-30 22:00:00,0.0,1.271205e-14,3.108624e-14,0.4175,1.727,4.7453,0.0,1.271205e-14,-8.215650e-15,3.108624e-14,-1.465494e-14,6.128431e-14,-1.19225,-1.19200,-1.19200,-1.19500,-1.195,-1.195,-1.1950
2017-09-30 23:00:00,0.0,1.271205e-14,3.108624e-14,0.2116,1.727,4.7453,0.0,1.271205e-14,-8.215650e-15,3.108624e-14,-1.465494e-14,6.128431e-14,-1.19200,-1.19200,-1.19500,-1.19500,-1.195,-1.195,-1.1965


In [12]:
df_train_0H = train_df[train_df.columns.difference(['Y.waterlevel.FC.1H', 'Y.waterlevel.FC.2H', 
                                                    'Y.waterlevel.FC.3H', 'Y.waterlevel.FC.4H', 
                                                    'Y.waterlevel.FC.5H', 'Y.waterlevel.FC.6H'])]
df_train_1H = train_df[train_df.columns.difference(['Y.waterlevel', 'Y.waterlevel.FC.2H', 
                                                    'Y.waterlevel.FC.3H', 'Y.waterlevel.FC.4H', 
                                                    'Y.waterlevel.FC.5H', 'Y.waterlevel.FC.6H'])]
df_train_2H = train_df[train_df.columns.difference(['Y.waterlevel', 'Y.waterlevel.FC.1H', 
                                                    'Y.waterlevel.FC.3H', 'Y.waterlevel.FC.4H', 
                                                    'Y.waterlevel.FC.5H', 'Y.waterlevel.FC.6H'])]
df_train_3H = train_df[train_df.columns.difference(['Y.waterlevel', 'Y.waterlevel.FC.1H', 
                                                    'Y.waterlevel.FC.2H', 'Y.waterlevel.FC.4H', 
                                                    'Y.waterlevel.FC.5H', 'Y.waterlevel.FC.6H'])]
df_train_4H = train_df[train_df.columns.difference(['Y.waterlevel', 'Y.waterlevel.FC.1H', 
                                                    'Y.waterlevel.FC.2H', 'Y.waterlevel.FC.3H', 
                                                    'Y.waterlevel.FC.5H', 'Y.waterlevel.FC.6H'])]
df_train_5H = train_df[train_df.columns.difference(['Y.waterlevel', 'Y.waterlevel.FC.1H', 
                                                    'Y.waterlevel.FC.2H', 'Y.waterlevel.FC.3H', 
                                                    'Y.waterlevel.FC.4H', 'Y.waterlevel.FC.6H'])]
df_train_6H = train_df[train_df.columns.difference(['Y.waterlevel', 'Y.waterlevel.FC.1H', 
                                                    'Y.waterlevel.FC.2H', 'Y.waterlevel.FC.3H', 
                                                    'Y.waterlevel.FC.4H', 'Y.waterlevel.FC.5H'])]
# df_train_2H.columns

df_test_0H = test_df[test_df.columns.difference(['Y.waterlevel.FC.1H', 'Y.waterlevel.FC.2H', 
                                                    'Y.waterlevel.FC.3H', 'Y.waterlevel.FC.4H', 
                                                    'Y.waterlevel.FC.5H', 'Y.waterlevel.FC.6H'])]
df_test_1H = test_df[test_df.columns.difference(['Y.waterlevel', 'Y.waterlevel.FC.2H', 
                                                    'Y.waterlevel.FC.3H', 'Y.waterlevel.FC.4H', 
                                                    'Y.waterlevel.FC.5H', 'Y.waterlevel.FC.6H'])]
df_test_2H = test_df[test_df.columns.difference(['Y.waterlevel', 'Y.waterlevel.FC.1H', 
                                                    'Y.waterlevel.FC.3H', 'Y.waterlevel.FC.4H', 
                                                    'Y.waterlevel.FC.5H', 'Y.waterlevel.FC.6H'])]
df_test_3H = test_df[test_df.columns.difference(['Y.waterlevel', 'Y.waterlevel.FC.1H', 
                                                    'Y.waterlevel.FC.2H', 'Y.waterlevel.FC.4H', 
                                                    'Y.waterlevel.FC.5H', 'Y.waterlevel.FC.6H'])]
df_test_4H = test_df[test_df.columns.difference(['Y.waterlevel', 'Y.waterlevel.FC.1H', 
                                                    'Y.waterlevel.FC.2H', 'Y.waterlevel.FC.3H', 
                                                    'Y.waterlevel.FC.5H', 'Y.waterlevel.FC.6H'])]
df_test_5H = test_df[test_df.columns.difference(['Y.waterlevel', 'Y.waterlevel.FC.1H', 
                                                    'Y.waterlevel.FC.2H', 'Y.waterlevel.FC.3H', 
                                                    'Y.waterlevel.FC.4H', 'Y.waterlevel.FC.6H'])]
df_test_6H = test_df[test_df.columns.difference(['Y.waterlevel', 'Y.waterlevel.FC.1H', 
                                                    'Y.waterlevel.FC.2H', 'Y.waterlevel.FC.3H', 
                                                    'Y.waterlevel.FC.4H', 'Y.waterlevel.FC.5H'])]

In [13]:
df_train_6H.tail()

,X.coming1Hsum,X.coming2Hsum,X.coming3Hsum,X.coming4Hsum,X.coming5Hsum,X.coming6Hsum,X.last16Hsum,X.last1Hsum,X.last2Hsum,X.last32Hsum,X.last4Hsum,X.last8Hsum,Y.waterlevel.FC.6H
datetime,,,,,,,,,,,,,
2017-09-30 19:00:00,0.0,1.271205e-14,-8.215650e-15,3.108624e-14,-1.465494e-14,6.128431e-14,1.727,0.0,1.271205e-14,4.7453,2.116000e-01,1.4026,-1.1950
2017-09-30 20:00:00,0.0,1.271205e-14,-8.215650e-15,3.108624e-14,-1.465494e-14,6.128431e-14,1.727,0.0,1.271205e-14,4.7453,3.108624e-14,1.0244,-1.1950
2017-09-30 21:00:00,0.0,1.271205e-14,-8.215650e-15,3.108624e-14,-1.465494e-14,6.128431e-14,1.727,0.0,1.271205e-14,4.7453,3.108624e-14,0.8505,-1.1950
2017-09-30 22:00:00,0.0,1.271205e-14,-8.215650e-15,3.108624e-14,-1.465494e-14,6.128431e-14,1.727,0.0,1.271205e-14,4.7453,3.108624e-14,0.4175,-1.1950
2017-09-30 23:00:00,0.0,1.271205e-14,-8.215650e-15,3.108624e-14,-1.465494e-14,6.128431e-14,1.727,0.0,1.271205e-14,4.7453,3.108624e-14,0.2116,-1.1965


In [14]:
def nn_waterstand(train_df, test_df):
    
    scaler = MinMaxScaler()
    scaler.fit(train_df)
    data_train = scaler.transform(train_df)
    data_test = scaler.transform(test_df)

    # Build X and y
    X_train = data_train[:,:-1]
    y_train = data_train[:, -1]
    X_test = data_test[:, :-1]
    y_test = data_test[:, -1]    
    
    # Number of stocks in training data
    n_params = X_train.shape[1]
    
    # Neurons
    n_neurons_1 = 1024
    n_neurons_2 = 512
    n_neurons_3 = 256
    n_neurons_4 = 128

    # Session
    net = tf.InteractiveSession()

    # Placeholder
    X = tf.placeholder(dtype=tf.float32, shape=[None, n_params])
    Y = tf.placeholder(dtype=tf.float32, shape=[None])

    # Initializers
    sigma = 1
    weight_initializer = tf.variance_scaling_initializer(mode="fan_avg", distribution="uniform", scale=sigma)
    bias_initializer = tf.zeros_initializer()

    # Hidden weights
    W_hidden_1 = tf.Variable(weight_initializer([n_params, n_neurons_1]))
    bias_hidden_1 = tf.Variable(bias_initializer([n_neurons_1]))
    W_hidden_2 = tf.Variable(weight_initializer([n_neurons_1, n_neurons_2]))
    bias_hidden_2 = tf.Variable(bias_initializer([n_neurons_2]))
    W_hidden_3 = tf.Variable(weight_initializer([n_neurons_2, n_neurons_3]))
    bias_hidden_3 = tf.Variable(bias_initializer([n_neurons_3]))
    W_hidden_4 = tf.Variable(weight_initializer([n_neurons_3, n_neurons_4]))
    bias_hidden_4 = tf.Variable(bias_initializer([n_neurons_4]))

    # Output weights
    W_out = tf.Variable(weight_initializer([n_neurons_4, 1]))
    bias_out = tf.Variable(bias_initializer([1]))

    # Hidden layer
    hidden_1 = tf.nn.relu(tf.add(tf.matmul(X, W_hidden_1), bias_hidden_1))
    hidden_2 = tf.nn.relu(tf.add(tf.matmul(hidden_1, W_hidden_2), bias_hidden_2))
    hidden_3 = tf.nn.relu(tf.add(tf.matmul(hidden_2, W_hidden_3), bias_hidden_3))
    hidden_4 = tf.nn.relu(tf.add(tf.matmul(hidden_3, W_hidden_4), bias_hidden_4))

    # Output layer (transpose!)
    out = tf.transpose(tf.add(tf.matmul(hidden_4, W_out), bias_out))

    # Cost function
    mse = tf.reduce_mean(tf.squared_difference(out, Y))

    # Optimizer
    opt = tf.train.AdamOptimizer().minimize(mse)

    # Init
    net.run(tf.global_variables_initializer())

    # Fit neural net
    batch_size = 256
    mse_train = []
    mse_test = []    

    %matplotlib
    # Setup plot
    plt.ion()
    fig = plt.figure()
    ax1 = fig.add_subplot(111)
    line1, = ax1.plot(scaler.inverse_transform(data_test)[:, -1])
    line2, = ax1.plot((scaler.inverse_transform(
        data_test)[:, -1]) * 0.5)

    # Run
    epochs = 10
    y_out = data_test.copy()
    for e in range(epochs):

        # Shuffle training data
        shuffle_indices = np.random.permutation(np.arange(len(y_train)))
        X_train = X_train[shuffle_indices]
        y_train = y_train[shuffle_indices]

        # Minibatch training
        for i in range(0, len(y_train) // batch_size):
            start = i * batch_size
            batch_x = X_train[start:start + batch_size]
            batch_y = y_train[start:start + batch_size]
            # Run optimizer with batch
            net.run(opt, feed_dict={X: batch_x, Y: batch_y})    
            # Show progress
            if np.mod(i, 50) == 0:
                # MSE train and test
                mse_train.append(net.run(mse, feed_dict={X: X_train, Y: y_train}))
                mse_test.append(net.run(mse, feed_dict={X: X_test, Y: y_test}))
                # Prediction
                pred = net.run(out, feed_dict={X: X_test})

                y_out[:, -1] = pred
                line2.set_ydata(scaler.inverse_transform(y_out)[:, -1])
                ax1.set_title('Epoch ' + str(e) + ', Batch ' + str(i) + ', MSE Train: ' +
                              str(mse_train[-1]) + ', MSE Test: ' + str(mse_test[-1]))

                fig.canvas.draw()  
    return net, out, X, X_test, y_test

In [15]:
net_0H, out_0H, X_0H, X_test_0H, y_test_0H = nn_waterstand(df_train_0H, df_test_0H)
net_1H, out_1H, X_1H, X_test_1H, y_test_1H = nn_waterstand(df_train_1H, df_test_1H)
net_2H, out_2H, X_2H, X_test_2H, y_test_2H = nn_waterstand(df_train_2H, df_test_2H)
net_3H, out_3H, X_3H, X_test_3H, y_test_3H = nn_waterstand(df_train_3H, df_test_3H)
net_4H, out_4H, X_4H, X_test_4H, y_test_4H = nn_waterstand(df_train_4H, df_test_4H)
net_5H, out_5H, X_5H, X_test_5H, y_test_5H = nn_waterstand(df_train_5H, df_test_5H)
net_6H, out_6H, X_6H, X_test_6H, y_test_6H = nn_waterstand(df_train_6H, df_test_6H)

Using matplotlib backend: TkAgg
Using matplotlib backend: TkAgg
Using matplotlib backend: TkAgg
Using matplotlib backend: TkAgg
Using matplotlib backend: TkAgg
Using matplotlib backend: TkAgg
Using matplotlib backend: TkAgg


In [16]:
pred_0H = net_0H.run(out_0H, feed_dict={X_0H: X_test_0H})
pred_1H = net_1H.run(out_1H, feed_dict={X_1H: X_test_1H})
pred_2H = net_2H.run(out_2H, feed_dict={X_2H: X_test_2H})
pred_3H = net_3H.run(out_3H, feed_dict={X_3H: X_test_3H})
pred_4H = net_4H.run(out_4H, feed_dict={X_4H: X_test_4H})
pred_5H = net_5H.run(out_5H, feed_dict={X_5H: X_test_5H})
pred_6H = net_6H.run(out_6H, feed_dict={X_6H: X_test_6H})

In [17]:
from_date = '2015-08-28 18:00'
to_date = '2015-09-02 08:00'

In [18]:
index_start = df_test_0H.index.get_loc(from_date)
dt_index = df_test_0H[from_date:to_date].index

In [19]:
start_setxlim = df_test_0H[from_date:to_date].index.to_pydatetime()[0]
end_setxlim = df_test_0H[from_date:to_date].index.to_pydatetime()[-1]

In [20]:
index_start

3594

In [21]:
#import matplotlib.dates as mdates
%matplotlib inline
df_band = pd.DataFrame(columns=['date', 'min_band', 'max_band'])
for idx, val in enumerate(dt_index):
    i_val = dt_index[idx]  # +1]
    i_end = i_val.strftime('%Y-%m-%d %H:%M')
    i_start = (i_val - pd.Timedelta(2, 'h')).strftime('%Y-%m-%d %H:%M')
    fig = plt.figure(figsize=(14, 3))
    ax = fig.add_subplot(111)

    # observed
    i_end = i_val.strftime('%Y-%m-%d %H:%M')
    i_start = (i_val - pd.Timedelta(2, 'h')).strftime('%Y-%m-%d %H:%M')
    line1, = ax.plot(df_test_0H[from_date:i_end].index.to_pydatetime(
    ), y_test_0H[index_start:index_start + idx + 1])  # , vmin=0.25, vmax=0.8)

    # 0H
    line2, = ax.plot(df_test_0H[i_start:i_end].index.to_pydatetime(), pred_0H.flatten()[
                     index_start + idx - 2:index_start + idx + 1], color='orange')

    # 1H
    i_val_1H = (i_val + pd.Timedelta(1, 'h'))
    i_end_1H = i_val_1H.strftime('%Y-%m-%d %H:%M')
    i_start_1H = (i_val_1H - pd.Timedelta(2, 'h')).strftime('%Y-%m-%d %H:%M')
    line3, = ax.plot(df_test_1H[i_start_1H:i_end_1H].index.to_pydatetime(
    ), pred_1H.flatten()[index_start + idx - 2:index_start + idx + 1])

    # 2H
    i_val_2H = (i_val + pd.Timedelta(2, 'h'))
    i_end_2H = i_val_2H.strftime('%Y-%m-%d %H:%M')
    i_start_2H = (i_val_2H - pd.Timedelta(3, 'h')).strftime('%Y-%m-%d %H:%M')
    line3, = ax.plot(df_test_2H[i_start_2H:i_end_2H].index.to_pydatetime(
    ), pred_2H.flatten()[index_start + idx - 3:index_start + idx + 1])

    # 3H
    i_val_3H = (i_val + pd.Timedelta(3, 'h'))
    i_end_3H = i_val_3H.strftime('%Y-%m-%d %H:%M')
    i_start_3H = (i_val_3H - pd.Timedelta(3, 'h')).strftime('%Y-%m-%d %H:%M')
    line3, = ax.plot(df_test_3H[i_start_3H:i_end_3H].index.to_pydatetime(
    ), pred_3H.flatten()[index_start + idx - 3:index_start + idx + 1])

    # 4H
    i_val_4H = (i_val + pd.Timedelta(4, 'h'))
    i_end_4H = i_val_4H.strftime('%Y-%m-%d %H:%M')
    i_start_4H = (i_val_4H - pd.Timedelta(3, 'h')).strftime('%Y-%m-%d %H:%M')
    line3, = ax.plot(df_test_4H[i_start_4H:i_end_4H].index.to_pydatetime(
    ), pred_4H.flatten()[index_start + idx - 3:index_start + idx + 1])

    # 5H
    i_val_5H = (i_val + pd.Timedelta(5, 'h'))
    i_end_5H = i_val_5H.strftime('%Y-%m-%d %H:%M')
    i_start_5H = (i_val_5H - pd.Timedelta(3, 'h')).strftime('%Y-%m-%d %H:%M')
    line3, = ax.plot(df_test_5H[i_start_5H:i_end_5H].index.to_pydatetime(
    ), pred_5H.flatten()[index_start + idx - 3:index_start + idx + 1])

    # 6H
    i_val_6H = (i_val + pd.Timedelta(6, 'h'))
    i_end_6H = i_val_6H.strftime('%Y-%m-%d %H:%M')
    i_start_6H = (i_val_6H - pd.Timedelta(3, 'h')).strftime('%Y-%m-%d %H:%M')
    line3, = ax.plot(df_test_6H[i_start_6H:i_end_6H].index.to_pydatetime(
    ), pred_6H.flatten()[index_start + idx - 3:index_start + idx + 1])

    ax.set_ylim(0.25, 1.1)
    ax.set_xlim(start_setxlim, end_setxlim)
    ax.axvline(x=i_val.to_pydatetime(), color=color_peilvak)

    for spine in ax.spines.values():
        spine.set_edgecolor(color_peilvak)
    #ax.format_xdata = mdates.DateFormatter('%Y-%m-%d %H')
    # fig.autofmt_xdate()

    band_now = np.concatenate([pred_6H.flatten()[index_start + idx - 3:index_start + idx + 1],
                               pred_5H.flatten()[
        index_start + idx - 3:index_start + idx + 1],
        pred_4H.flatten()[
        index_start + idx - 3:index_start + idx + 1],
        pred_3H.flatten()[
        index_start + idx - 3:index_start + idx + 1],
        pred_2H.flatten()[index_start + idx - 3:index_start + idx + 1],
        pred_1H.flatten()[index_start + idx - 2:index_start + idx + 1]])
    band_now_max = band_now.max()
    band_now_min = band_now.min()
    df_band.loc[idx] = [i_val, band_now_min, band_now_max]

    plt.fill_between(pd.Index(df_band['date']).to_pydatetime(
    ), df_band['min_band'].values, df_band['max_band'].values, color='lightgrey', alpha='0.5')
    #ax.fill_between(x, y3, y4, color='grey', alpha='0.5')
    file_png_out = os.path.join(
        png_out, 'datum_{}.png'.format(i_val.strftime('%Y%m%d%H%M')))
    print(file_png_out)
    plt.savefig(file_png_out)

    # plt.show()
    plt.close(fig)

D:\Projects\RO\DataChallenge2018\png_out\waterlevel.peilvak.92\datum_201508281800.png
D:\Projects\RO\DataChallenge2018\png_out\waterlevel.peilvak.92\datum_201508281900.png
D:\Projects\RO\DataChallenge2018\png_out\waterlevel.peilvak.92\datum_201508282000.png
D:\Projects\RO\DataChallenge2018\png_out\waterlevel.peilvak.92\datum_201508282100.png
D:\Projects\RO\DataChallenge2018\png_out\waterlevel.peilvak.92\datum_201508282200.png
D:\Projects\RO\DataChallenge2018\png_out\waterlevel.peilvak.92\datum_201508282300.png
D:\Projects\RO\DataChallenge2018\png_out\waterlevel.peilvak.92\datum_201508290000.png
D:\Projects\RO\DataChallenge2018\png_out\waterlevel.peilvak.92\datum_201508290100.png
D:\Projects\RO\DataChallenge2018\png_out\waterlevel.peilvak.92\datum_201508290200.png
D:\Projects\RO\DataChallenge2018\png_out\waterlevel.peilvak.92\datum_201508290300.png
D:\Projects\RO\DataChallenge2018\png_out\waterlevel.peilvak.92\datum_201508290400.png
D:\Projects\RO\DataChallenge2018\png_out\waterlevel.pe

D:\Projects\RO\DataChallenge2018\png_out\waterlevel.peilvak.92\datum_201509011800.png
D:\Projects\RO\DataChallenge2018\png_out\waterlevel.peilvak.92\datum_201509011900.png
D:\Projects\RO\DataChallenge2018\png_out\waterlevel.peilvak.92\datum_201509012000.png
D:\Projects\RO\DataChallenge2018\png_out\waterlevel.peilvak.92\datum_201509012100.png
D:\Projects\RO\DataChallenge2018\png_out\waterlevel.peilvak.92\datum_201509012200.png
D:\Projects\RO\DataChallenge2018\png_out\waterlevel.peilvak.92\datum_201509012300.png
D:\Projects\RO\DataChallenge2018\png_out\waterlevel.peilvak.92\datum_201509020000.png
D:\Projects\RO\DataChallenge2018\png_out\waterlevel.peilvak.92\datum_201509020100.png
D:\Projects\RO\DataChallenge2018\png_out\waterlevel.peilvak.92\datum_201509020200.png
D:\Projects\RO\DataChallenge2018\png_out\waterlevel.peilvak.92\datum_201509020300.png
D:\Projects\RO\DataChallenge2018\png_out\waterlevel.peilvak.92\datum_201509020400.png
D:\Projects\RO\DataChallenge2018\png_out\waterlevel.pe